In [114]:
import os
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [115]:
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

from keras import backend as K
K.set_image_dim_ordering('tf')

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop,adam

In [116]:
%matplotlib inline
df_train = pd.read_json('./data_raw/train.json')
df_test = pd.read_json('./data_raw/test.json')

In [117]:
df_train['label_id']=df_train["annotations"].apply(lambda x: x["label_id"])
df_train['img_id']=df_train["annotations"].apply(lambda x: x["image_id"])
df_train['url']=df_train["images"].apply(lambda x: x["url"][0])

In [119]:
df_train_shuff = df_train.sample(frac=1).reset_index(drop=True)

In [123]:
type(df_train.loc[0]["label_id"])

numpy.int64

In [ ]:
img_data_list=[]
img_rows=128
img_cols=128
img_label =[]
for idx, row in df_train_shuff.loc[0:1000].iterrows():
    try:
        resp = urllib.request.urlopen(str(row["url"]))
        image = np.asarray(bytearray(resp.read()), dtype="uint8")
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        print(str(idx)+' - ')
        #input_img = url_to_image(row["url"])
        image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        input_img = cv2.resize(image,(img_rows,img_cols))
        img_data_list.append(input_img)
        img_label.append(row["label_id"])
    except:
        pass
    

0

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

16

17

18

19

20

21

22

23

24

25

26

27

28

29

30

31

32

33

34

35

36

37

38

39

40

41

42

44

45

46

47

48

49

50

51

52

53

54

55

56

57

58

60

61

62

64

65

66

67

68

69

70

71

72

73

74

75

76

77

78

79

80

81

82

83

84

85

86

87

88

89

91

93

94

95

96

97

98

100

101

102

103

104

105

106

107

108

109

110

111

112

113

114

115

116

117

118

119

120

121

122

123

124

125

126

127

128

129

130

131

132

133

134

135

136

137

138

139

140

141

142

143

145

146

147

148

149

150

152

153

154

155

156

157

158

159

160

161

162

163

164

165

166

167

168

169

170

171

173

174

175

176

177

178

179

180

181

182

183

184

185

186

187

188

189

190

191

192

193

194

196

197

198

199

200

201

202

203

204

205

206

207

208

209

210

211

212

213

214

217

218

219

221

222

223

224

225

227

228

229

230

231

232

233

234


In [59]:
img_data = np.array(img_data_list)

In [87]:
y=np.array(df_train_shuff.loc[0:10]["label_id"].tolist())

In [ ]:
img_data.shape
y.shape

In [98]:
df_dum = pd.get_dummies(df_train_shuff.loc[0:10]["label_id"].tolist())
df_dum = pd.DataFrame(df_dum)
Y = df_dum.as_matrix()
Y.shape

(11, 11)

In [101]:
img_data = img_data/255

In [105]:
img_data.max()

1.0

In [109]:
img_data.shape

(11, 128, 128, 1)

In [107]:
Y.shape[0]

(11, 11)

In [108]:
img_data= np.expand_dims(img_data, axis=4) 

In [111]:
input_shape=img_data[0].shape
print(input_shape)
model = Sequential()

model.add(Convolution2D(32, 3,3,border_mode='same',input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
#model.add(Convolution2D(64, 3, 3))
#model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(Y.shape[1]))
model.add(Activation('softmax'))

#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=["accuracy"])
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=["accuracy"])


(128, 128, 1)


/home/anto/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(128, 128,..., padding="same")`
  """
/home/anto/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  import sys
/home/anto/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  if sys.path[0] == '':


In [113]:
hist = model.fit(img_data, Y, nb_epoch=25, verbose=1)

/home/anto/anaconda3/lib/python3.6/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/25
11/11 [==============================] - 0s - loss: 2.4258 - acc: 0.2727
Epoch 2/25
11/11 [==============================] - 1s - loss: 2.2801 - acc: 0.2727
Epoch 3/25
11/11 [==============================] - 1s - loss: 2.3109 - acc: 0.1818
Epoch 4/25
11/11 [==============================] - 0s - loss: 2.3735 - acc: 0.1818
Epoch 5/25
11/11 [==============================] - 1s - loss: 2.3423 - acc: 0.0909
Epoch 6/25
11/11 [==============================] - 1s - loss: 2.2752 - acc: 0.2727
Epoch 7/25
11/11 [==============================] - 0s - loss: 2.0757 - acc: 0.4545
Epoch 8/25
11/11 [==============================] - 1s - loss: 2.7860 - acc: 0.2727
Epoch 9/25
11/11 [==============================] - 1s - loss: 2.2373 - acc: 0.2727
Epoch 10/25
11/11 [==============================] - 0s - loss: 2.1026 - acc: 0.3636
Epoch 11/25
11/11 [==============================] - 0s - loss: 2.1429 - acc: 0.2727
Epoch 12/25
11/11 [==============================] - 1s - loss: 2.0286 - a